In [1]:
!git clone https://github.com/OmarMoMorgan/DL_init.git

Cloning into 'DL_init'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 68 (delta 23), reused 57 (delta 12), pack-reused 0
Unpacking objects: 100% (68/68), 24.80 KiB | 1.30 MiB/s, done.


In [2]:
from torchvision.datasets import CIFAR10
from  torch.utils.data import DataLoader,Dataset
from torch import nn , optim
import torch
import pandas as pd


In [3]:
import sys
sys.path.append('/kaggle/working/DL_init')
from models import build_model
from tools import EarlyStopping, tune_model
from data import train_test_split_ , perPixel_mean_std, perChannel_mean_std, build_transforms

In [4]:
class MyDataset(Dataset):
    def __init__(self,x,y,transforms = None):
        self.x = x
        self.y = y
        self.transforms = transforms
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        if self.transforms is not None:
            x= self.transforms(image= self.x[idx])['image']
            return x.type(torch.float32),self.y[idx]

        return self.x[idx],self.y[idx]

In [5]:
train_data  = CIFAR10(root='./data',train = True,download = True)
test_data = CIFAR10(root='/data',train=False,download = True)


100%|██████████| 170498071/170498071 [00:02<00:00, 84868540.79it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


100%|██████████| 170498071/170498071 [00:02<00:00, 57382753.22it/s]


Extracting /data/cifar-10-python.tar.gz to /data


In [6]:
# make a validation set
config= {'random_state' : 42,
          'shuffle':True,
         'train_size':0.9 }
X_train, X_val, y_train, y_val= train_test_split_(train_data.data,train_data.targets,config)
X_train.shape,X_val.shape

((45000, 32, 32, 3), (5000, 32, 32, 3))

```python
# stratify to keep the ratio 0.9 for every class in the trainined and testing
import numpy as np
uni, counts = np.unique(y_val, return_counts=True)
counts
```

In [7]:
mean_,std_ = perChannel_mean_std(X_train,'NHWC')
mean_,std_

(array([125.24002457, 122.93835269, 113.84449184]),
 array([62.95162973, 62.07468216, 66.70886545]))

In [8]:
train_tansforms,test_transforms = build_transforms('pipeline1',{'mean':mean_,
                                                                 'std':std_})

In [9]:
trainset = MyDataset(X_train,y_train,train_tansforms)
trainloader_ = DataLoader(trainset,batch_size=128, shuffle=True)

valset = MyDataset(X_val,y_val,test_transforms)
valloader_ = DataLoader(trainset,batch_size=1024, shuffle=False)

testset = MyDataset(test_data.data,test_data.targets,test_transforms)
testloader_ = DataLoader(testset,batch_size=512, shuffle=False)

In [10]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## For n = 5 groups = 1 for the first arch

In [11]:
model = build_model('sub_CNN',[5,5,5],1,[16,32,62])
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [12]:
hist_= tune_model(100,model,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

For epoch   4 || Training Loss 0.683 || acc 0.759 || Testing Loss 0.670 || Test acc 0.766
For epoch   9 || Training Loss 0.482 || acc 0.833 || Testing Loss 0.472 || Test acc 0.837
For epoch  14 || Training Loss 0.388 || acc 0.865 || Testing Loss 0.370 || Test acc 0.872
For epoch  19 || Training Loss 0.323 || acc 0.888 || Testing Loss 0.336 || Test acc 0.883
For epoch  24 || Training Loss 0.274 || acc 0.904 || Testing Loss 0.264 || Test acc 0.909
For epoch  29 || Training Loss 0.236 || acc 0.918 || Testing Loss 0.213 || Test acc 0.928
For epoch  34 || Training Loss 0.203 || acc 0.928 || Testing Loss 0.183 || Test acc 0.936
For epoch  39 || Training Loss 0.182 || acc 0.936 || Testing Loss 0.166 || Test acc 0.941
For epoch  44 || Training Loss 0.165 || acc 0.942 || Testing Loss 0.193 || Test acc 0.932
Early Stopping Activated


In [13]:
print('best loss is',earlystopping.best_loss)
torch.save(earlystopping.best_model, './mod_res5_g1')

try:
    df = pd.DataFrame.from_dict(hist_)
    df.to_csv('mod_res3_g1.csv', index=False)

except:
    print('failed')



best loss is 0.15825225175781685


## For n = 5 groups = 4 second arch

In [14]:
model = build_model('sub_CNN',[5,5,5],4,[16,32,64])
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [15]:
hist_= tune_model(100,model,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

For epoch   4 || Training Loss 0.909 || acc 0.684 || Testing Loss 0.981 || Test acc 0.662
For epoch   9 || Training Loss 0.736 || acc 0.745 || Testing Loss 0.707 || Test acc 0.754
For epoch  14 || Training Loss 0.645 || acc 0.776 || Testing Loss 0.657 || Test acc 0.772
For epoch  19 || Training Loss 0.588 || acc 0.796 || Testing Loss 0.565 || Test acc 0.803
For epoch  24 || Training Loss 0.532 || acc 0.815 || Testing Loss 0.537 || Test acc 0.812
For epoch  29 || Training Loss 0.492 || acc 0.829 || Testing Loss 0.504 || Test acc 0.821
For epoch  34 || Training Loss 0.466 || acc 0.837 || Testing Loss 0.464 || Test acc 0.839
Early Stopping Activated


In [16]:
print('best loss is',earlystopping.best_loss)
torch.save(earlystopping.best_model, './mod_res5_g4')

try:
    df = pd.DataFrame.from_dict(hist_)
    df.to_csv('mod_res5_g4.csv', index=False)

except:
    print('failed')



best loss is 0.45523006265813654


## for n = 5 wider filters third arch

In [17]:
model = build_model('sub_CNN',[5,5,5],3,[33,63,129])
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [18]:
hist_= tune_model(100,model,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

For epoch   4 || Training Loss 0.743 || acc 0.740 || Testing Loss 0.700 || Test acc 0.757
For epoch   9 || Training Loss 0.546 || acc 0.811 || Testing Loss 0.525 || Test acc 0.819
For epoch  14 || Training Loss 0.437 || acc 0.847 || Testing Loss 0.413 || Test acc 0.859
For epoch  19 || Training Loss 0.362 || acc 0.875 || Testing Loss 0.383 || Test acc 0.866
For epoch  24 || Training Loss 0.303 || acc 0.895 || Testing Loss 0.364 || Test acc 0.873
For epoch  29 || Training Loss 0.263 || acc 0.908 || Testing Loss 0.260 || Test acc 0.908
For epoch  34 || Training Loss 0.230 || acc 0.920 || Testing Loss 0.276 || Test acc 0.906
For epoch  39 || Training Loss 0.203 || acc 0.928 || Testing Loss 0.216 || Test acc 0.925
For epoch  44 || Training Loss 0.180 || acc 0.937 || Testing Loss 0.181 || Test acc 0.936
For epoch  49 || Training Loss 0.167 || acc 0.941 || Testing Loss 0.154 || Test acc 0.946
For epoch  54 || Training Loss 0.148 || acc 0.948 || Testing Loss 0.149 || Test acc 0.948
Early Stop

In [19]:
print('best loss is',earlystopping.best_loss)
torch.save(earlystopping.best_model, './mod_res5_g4_wide')

try:
    df = pd.DataFrame.from_dict(hist_)
    df.to_csv('mod_res5_g4_wide.csv', index=False)

except:
    print('failed')

best loss is 0.1318205495110967


## for n = 7 wider arch model 4

In [20]:
model = build_model('sub_CNN',[7,7,7],4,[32,64,128])
model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [21]:
hist_= tune_model(100,model,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

For epoch   4 || Training Loss 0.793 || acc 0.723 || Testing Loss 0.763 || Test acc 0.735
For epoch   9 || Training Loss 0.594 || acc 0.793 || Testing Loss 0.659 || Test acc 0.773
For epoch  14 || Training Loss 0.493 || acc 0.830 || Testing Loss 0.473 || Test acc 0.834
For epoch  19 || Training Loss 0.414 || acc 0.856 || Testing Loss 0.407 || Test acc 0.858
For epoch  24 || Training Loss 0.360 || acc 0.874 || Testing Loss 0.396 || Test acc 0.860
For epoch  29 || Training Loss 0.311 || acc 0.892 || Testing Loss 0.305 || Test acc 0.894
For epoch  34 || Training Loss 0.274 || acc 0.905 || Testing Loss 0.321 || Test acc 0.886
For epoch  39 || Training Loss 0.240 || acc 0.916 || Testing Loss 0.231 || Test acc 0.920
For epoch  44 || Training Loss 0.215 || acc 0.925 || Testing Loss 0.230 || Test acc 0.920
For epoch  49 || Training Loss 0.198 || acc 0.929 || Testing Loss 0.190 || Test acc 0.934
For epoch  54 || Training Loss 0.179 || acc 0.938 || Testing Loss 0.174 || Test acc 0.940
Early Stop

In [22]:
print('best loss is',earlystopping.best_loss)
torch.save(earlystopping.best_model, './sub_cnn_4')

try:
    df = pd.DataFrame.from_dict(hist_)
    df.to_csv('sub_cnn_4.csv', index=False)

except:
    print('failed')

best loss is 0.15574116869406265


# fine tunning
in this part we shall try to fine tune a model based on a already pre trained model\
I will use for this part **Effiecent net** \
part 1: freeze all layers excpet the last one 

In [23]:
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
weights = EfficientNet_B3_Weights.DEFAULT
model_freeze = efficientnet_b3(weights=weights)
for param in model_freeze.parameters():
    param.requires_grad = False
#model_freeze.classifier[1] = nn.Linear(1536,10)
model_freeze.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.3, inplace=True), 
    torch.nn.Linear(1536,10))
print(model_freeze)
#print(model_freeze.classifier[1].weight.requires_grad)

Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth
100%|██████████| 47.2M/47.2M [00:00<00:00, 144MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [24]:

model_freeze.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_freeze.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [25]:
hist_= tune_model(100,model_freeze,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

For epoch   4 || Training Loss 2.102 || acc 0.255 || Testing Loss 2.419 || Test acc 0.285
For epoch   9 || Training Loss 2.106 || acc 0.254 || Testing Loss 2.638 || Test acc 0.284
Early Stopping Activated


part 2: update all the layers 

In [26]:
weights = EfficientNet_B3_Weights.DEFAULT
model_free = efficientnet_b3(weights=weights)
model_free.classifier[1] = nn.Linear(1536,10)
model_free.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_free.parameters(),lr= 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min',patience = 5,factor = 0.1,verbose=True)
earlystopping = EarlyStopping()

In [27]:
hist_= tune_model(100,model_free,trainloader_,valloader_,\
               loss,optimizer,device,scheduler,earlystopping=earlystopping)

For epoch   4 || Training Loss 0.522 || acc 0.823 || Testing Loss 0.392 || Test acc 0.866
For epoch   9 || Training Loss 0.355 || acc 0.879 || Testing Loss 0.272 || Test acc 0.909
For epoch  14 || Training Loss 0.284 || acc 0.902 || Testing Loss 0.189 || Test acc 0.935
For epoch  19 || Training Loss 0.224 || acc 0.925 || Testing Loss 0.136 || Test acc 0.954
For epoch  24 || Training Loss 0.210 || acc 0.930 || Testing Loss 0.144 || Test acc 0.951
For epoch  29 || Training Loss 0.171 || acc 0.944 || Testing Loss 0.095 || Test acc 0.968
For epoch  34 || Training Loss 0.134 || acc 0.954 || Testing Loss 0.076 || Test acc 0.974
For epoch  39 || Training Loss 0.131 || acc 0.957 || Testing Loss 0.072 || Test acc 0.975
For epoch  44 || Training Loss 0.145 || acc 0.953 || Testing Loss 0.096 || Test acc 0.968
For epoch  49 || Training Loss 0.099 || acc 0.967 || Testing Loss 0.063 || Test acc 0.979
For epoch  54 || Training Loss 0.090 || acc 0.970 || Testing Loss 0.052 || Test acc 0.982
For epoch 